In [ ]:
# Import python packages
import streamlit as st

# Import Snowpark packages
from snowflake.snowpark.context import get_active_session
snowflake_session = get_active_session()

In [ ]:
use schema "DEMO";

# Metadata-driven ingestion

We now have all the tools we need to ingest data into Snowflake without manually defining any metadata.

First, these table templates can be used directly inside a `create table` statement:

```sql
create or replace table "MY_TABLE"
  using template(
    select array_agg(object_construct(*))
    from table(
      infer_schema(
          location => '<location of file(s), including stage>'
        , file_format => '<Snowflake File Format object to use when reading the file>'
      )
    )
  )
  comment = 'Table created using the metadata inferred from the source file(s)'
```

Once we have created our landing tables (whether using a template or by manually defining appropriate fields), data can be ingested:

```sql
copy into "MY_TABLE"
from '@<location of file(s), including stage>'
  file_format = "<Snowflake File Format object to use when reading the file>"
  match_by_column_name = CASE_INSENSITIVE
```

The key option here is "match_by_column_name":

- Removes the need for fields in the file to be in the same order as in the destination table
- Most useful for CSVs where field order used to be far more important
- Allows files to be ingested even if they are missing fields

## Metadata-driven ingestion - CSV

Let's quickly create a table using the table template from our example CSV file, then ingest the data:

In [ ]:
create or replace table "DATA_FROM_CSV"
  using template(
    select array_agg(object_construct(*))
    from table(
      infer_schema(
          location => '@"STG__DATA"/csv'
        , file_format => '"FF_CSV"'
      )
    )
  )
  comment = 'Table created using the metadata inferred from the source file(s) in CSV format'
;

desc table "DATA_FROM_CSV";

In [ ]:
copy into "DATA_FROM_CSV"
from '@"STG__DATA"/csv'
  file_format = "FF_CSV"
  match_by_column_name = CASE_INSENSITIVE

In [ ]:
select * from "DATA_FROM_CSV"

## Metadata-driven ingestion - JSON

And now let's create a table using the table template from our example JSON file, then ingest the data:

In [ ]:
create or replace table "DATA_FROM_JSON"
  using template(
    select array_agg(object_construct(*))
    from table(
      infer_schema(
          location => '@"STG__DATA"/json'
        , file_format => '"FF_JSON"'
      )
    )
  )
  comment = 'Table created using the metadata inferred from the source file(s) in JSON format'
;

desc table "DATA_FROM_JSON";

In [ ]:
copy into "DATA_FROM_JSON"
from '@"STG__DATA"/json'
  file_format = "FF_JSON"
  match_by_column_name = CASE_INSENSITIVE

In [ ]:
select * from "DATA_FROM_JSON"

## Metadata-driven ingestion - Parquet

Finally, let's create a table using the table template from our example Parquet file, then ingest the data:

In [ ]:
create or replace table "DATA_FROM_PARQUET"
  using template(
    select array_agg(object_construct(*))
    from table(
      infer_schema(
          location => '@"STG__DATA"/parquet'
        , file_format => '"FF_PARQUET"'
      )
    )
  )
  comment = 'Table created using the metadata inferred from the source file(s) in Parquet format'
;

desc table "DATA_FROM_PARQUET";

In [ ]:
copy into "DATA_FROM_PARQUET"
from '@"STG__DATA"/parquet'
  file_format = "FF_PARQUET"
  match_by_column_name = CASE_INSENSITIVE

In [ ]:
select * from "DATA_FROM_PARQUET"